### GX Core Workflow - Using Customised SQL Expectation

- Import Required Libraries
- Create GX Context (Ephemeral)
- Create DataSource (Sqlite DB)
- Create Data Asset (Sqlite Table)
- Create Batch Definition 
- Define Custom SQL Expectations
- Create SQL Expectation
- Create Expectations Suite and Add expectations
- Create Validation Definition (Expectation Suite, Batch Definition)
- Create Checkpoint (Validation Definition)
- Define Runtime Expectation parameters
- Run Checkpoint(Expectations parameters)
- Print Result

### Import Libraries

In [1]:
import great_expectations as gx


### Creating GX Context

In [2]:
context = gx.get_context(mode="ephemeral")


### Creating Datasource 

In [3]:
datasource_name = "sqlite_datasource"
data_source = context.data_sources.add_sqlite(
    name=datasource_name,
    connection_string="sqlite:///temperature.db"
)


### Creating Data Asset

In [4]:
data_asset_name = 'temperature_reading'
data_asset = data_source.add_table_asset(name = data_asset_name, table_name ='temperature_reading')


### Creating Batch Definition

In [5]:
batch_definition_name ='full_batch'
batch_definition = data_asset.add_batch_definition_whole_table(batch_definition_name)


### Define Customised Expectation class

In [6]:
from pydantic import Field
class ExpectValidTemperatureRange(gx.expectations.UnexpectedRowsExpectation):  
    unexpected_rows_query: str = Field(None, description="Query to find rows with unexpected values")
    column: str = Field(None, description="Column name")
    min_value: str = Field(None, description="Min value")
    max_value: str = Field(None, description="Max value")
    description = "Temperature values are within the specified normal range."
    def __init__(self, column, min_value, max_value, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.column = column
        self.min_value = min_value
        self.max_value = max_value
        self.unexpected_rows_query = (
            f"SELECT * FROM {{batch}} WHERE {self.column} < {self.min_value} OR {self.column} > {self.max_value}"
        )


In [7]:
expectation_temp = ExpectValidTemperatureRange(column="temperature", min_value=30, max_value=45)


### Creating Expectation Suite and Adding expectations

In [8]:
expectation_suite_name = "temperature_data_suite"
expectation_suite_ref = gx.ExpectationSuite(name=expectation_suite_name)
expectation_suite = context.suites.add(expectation_suite_ref)


In [9]:
expectation_suite.add_expectation(expectation_temp)


ExpectValidTemperatureRange(id='2766dd78-36c5-4e8b-939d-be4983faad69', meta=None, notes=None, result_format=<ResultFormat.BASIC: 'BASIC'>, description='Temperature values are within the specified normal range.', catch_exceptions=False, rendered_content=None, windows=None, batch_id=None, unexpected_rows_query='SELECT * FROM {batch} WHERE temperature < 30 OR temperature > 45', column='temperature', min_value=30, max_value=45)

### Creating Validation Defintions

In [10]:
validation_def_name = "temperature_data_validation"
validation_definition_ref = gx.ValidationDefinition( data=batch_definition,
                                             suite=expectation_suite,
                                             name=validation_def_name)


In [11]:
validation_definition = context.validation_definitions.add(validation_definition_ref)


### Creating Checkpoint 

In [12]:
checkpoint_name = "temperature_run_checkpoint"
checkpoint_to_add = gx.Checkpoint(name=checkpoint_name,                                 
                                  validation_definitions=[validation_definition],
                                  actions=[],
                                  result_format="BASIC"
)


In [13]:
checkpoint = context.checkpoints.add(checkpoint_to_add)


### Running Checkpoint 

In [14]:
validation_results = checkpoint.run()


Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

### Printing Result

In [15]:
print(validation_results)


run_id={"run_name": null, "run_time": "2025-06-23T17:46:25.432289+06:00"} run_results={ValidationResultIdentifier::temperature_data_suite/__none__/20250623T114625.432289Z/sqlite_datasource-temperature_reading: {
  "success": false,
  "results": [
    {
      "success": false,
      "expectation_config": {
        "type": "expect_valid_temperature_range",
        "kwargs": {
          "batch_id": "sqlite_datasource-temperature_reading",
          "unexpected_rows_query": "SELECT * FROM {batch} WHERE temperature < 30 OR temperature > 45",
          "column": "temperature",
          "min_value": 30,
          "max_value": 45
        },
        "meta": {},
        "id": "2766dd78-36c5-4e8b-939d-be4983faad69"
      },
      "result": {
        "observed_value": 7,
        "details": {
          "unexpected_rows": [
            {
              "reading_date": "2024-08-01",
              "city": "Mumbai",
              "temperature": 28
            },
            {
              "reading_dat